<a href="https://colab.research.google.com/github/Nikitha1418/INFO5731/blob/master/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 11/11/2020)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/INFO5731_FALL2020/blob/master/In_class_exercise/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
import pandas as pd
import nltk
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, KFold, cross_val_score
from xgboost import XGBClassifier

In [ ]:


train_df=pd.read_fwf("/content/stsa-train.txt", header=None)
train_df= pd.DataFrame(my_data_train)
train_df = train_df.rename(columns={0: "Sentiment", 1: "Text"})
test_df = pd.read_fwf("/content/stsa-test.txt", header=None)
test_df =  pd.DataFrame(my_data_test)
test_df = test_df.rename(columns={0: "Sentiment", 1: "Text"})


In [ ]:
#Removal of special characters:

train_df['Cleaned_Data'] = train_df['Text'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))

#Removal of punctuations :

train_df['Cleaned_Data'] = train_df['Cleaned_Data'].str.replace('[^\w\s]','')

train_df['Cleaned_Data'] = train_df['Cleaned_Data'].apply(lambda x :x.lower())

#Removing stopwords by using stopwordslist

stop = stopwords.words('english')
train_df['Cleaned_Data'] = train_df['Cleaned_Data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
# from textblob import TextBlob
nltk.download('punkt')
train_df['Cleaned_Data'] = train_df['Cleaned_Data'].apply(lambda x: TextBlob(x).words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from textblob import Word
import nltk
nltk.download('wordnet')
train_df['Cleaned_Data'] = train_df['Cleaned_Data'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
train_tfidf_vect = TfidfVectorizer(analyzer = 'word')
X_data = train_tfidf_vect.fit_transform(train_df['Cleaned_Data'])

In [ ]:
x_train_data, x_validation_data, y_train_data, y_validation_data = train_test_split(X_data, train_df['Sentiment'],test_size=0.2)

In [ ]:
test_df['Cleaned_Data'] = test_df['Text'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))

#Removal of punctuations :

test_df['Cleaned_Data'] = test_df['Cleaned_Data'].str.replace('[^\w\s]','')

test_df['Cleaned_Data'] = test_df['Cleaned_Data'].apply(lambda x : x.lower())

#Removing stopwords by using stopwordslist
stop = stopwords.words('english')
test_df['Cleaned_Data'] = test_df['Cleaned_Data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
from textblob import TextBlob
nltk.download('punkt')
test_df['Cleaned_Data'] = test_df['Cleaned_Data'].apply(lambda x: TextBlob(x).words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from textblob import Word
import nltk
nltk.download('wordnet')
test_df['Cleaned_Data'] = test_df['Cleaned_Data'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
test_tfidf_vect1 = TfidfVectorizer(analyzer = 'word', vocabulary = train_tfidf_vect.vocabulary_)
test_tfidf_vect1.fit(test_df['Cleaned_Data'])
x_test_data = test_tfidf_vect1.transform(test_df['Cleaned_Data'])
y_test_data = test_df['Sentiment']

In [ ]:
#Multinomial-Model
multinomial_model = MultinomialNB()
multinomial_model.fit(x_train_data, y_train_data)
validation_predicted_mnb = multinomial_model.predict(x_validation_data)
validation_accuracy_mnb = round(accuracy_score(validation_predicted_mnb,y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_mnb)
print(classification_report(y_validation_data, validation_predicted_mnb))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(multinomial_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_mnb = multinomial_model.predict(x_test_data)
test_accuracy_mnb = round(accuracy_score(test_predicted_mnb, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_mnb)
print(classification_report(y_test_data, test_predicted_mnb))

Validation Data
Accuracy -  77.53
              precision    recall  f1-score   support

           0       0.85      0.67      0.75       695
           1       0.73      0.88      0.80       689

    accuracy                           0.78      1384
   macro avg       0.79      0.78      0.77      1384
weighted avg       0.79      0.78      0.77      1384

Corss Validation Score -  72.105
Testing Data
Accuracy -  78.75
              precision    recall  f1-score   support

           0       0.86      0.68      0.76       912
           1       0.74      0.89      0.81       909

    accuracy                           0.79      1821
   macro avg       0.80      0.79      0.79      1821
weighted avg       0.80      0.79      0.79      1821



In [ ]:
#svm-Model

svm_model = LinearSVC()
svm_model.fit(x_train_data, y_train_data)
validation_predicted_svm = svm_model.predict(x_validation_data)
validation_accuracy_svm = round(accuracy_score(validation_predicted_svm,y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_svm)
print(classification_report(y_validation_data, validation_predicted_svm))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(svm_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_svm = svm_model.predict(x_test_data)
test_accuracy_svm = round(accuracy_score(test_predicted_svm, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_svm)
print(classification_report(y_test_data, test_predicted_svm))

Validation Data
Accuracy -  76.73
              precision    recall  f1-score   support

           0       0.78      0.74      0.76       695
           1       0.75      0.80      0.77       689

    accuracy                           0.77      1384
   macro avg       0.77      0.77      0.77      1384
weighted avg       0.77      0.77      0.77      1384

Corss Validation Score -  70.568
Testing Data
Accuracy -  78.47
              precision    recall  f1-score   support

           0       0.81      0.75      0.78       912
           1       0.77      0.82      0.79       909

    accuracy                           0.78      1821
   macro avg       0.79      0.78      0.78      1821
weighted avg       0.79      0.78      0.78      1821



In [ ]:
#KNN-Model

knn_model = KNeighborsClassifier(n_neighbors = 10)
knn_model.fit(x_train_data, y_train_data)
validation_predicted_knn = knn_model.predict(x_validation_data)
validation_accuracy_knn = round(accuracy_score(validation_predicted_knn, y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_knn)
print(classification_report(y_validation_data, validation_predicted_knn))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(knn_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_knn = knn_model.predict(x_test_data)
test_accuracy_knn = round(accuracy_score(test_predicted_knn, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_knn)
print(classification_report(y_test_data, test_predicted_knn))

Validation Data
Accuracy -  61.56
              precision    recall  f1-score   support

           0       0.57      0.91      0.70       695
           1       0.77      0.32      0.45       689

    accuracy                           0.62      1384
   macro avg       0.67      0.61      0.58      1384
weighted avg       0.67      0.62      0.58      1384

Corss Validation Score -  54.638
Testing Data
Accuracy -  60.85
              precision    recall  f1-score   support

           0       0.57      0.91      0.70       912
           1       0.77      0.30      0.44       909

    accuracy                           0.61      1821
   macro avg       0.67      0.61      0.57      1821
weighted avg       0.67      0.61      0.57      1821



In [ ]:
#DecisionTree

dt_model = DecisionTreeClassifier()
dt_model.fit(x_train_data, y_train_data)
validation_predicted_dt = dt_model.predict(x_validation_data)
validation_accuracy_dt = round(accuracy_score(validation_predicted_dt, y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_dt)
print(classification_report(y_validation_data, validation_predicted_dt))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(dt_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_dt = dt_model.predict(x_test_data)
test_accuracy_dt = round(accuracy_score(test_predicted_dt, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_dt)
print(classification_report(y_test_data, test_predicted_dt))

Validation Data
Accuracy -  65.97
              precision    recall  f1-score   support

           0       0.65      0.68      0.67       695
           1       0.67      0.64      0.65       689

    accuracy                           0.66      1384
   macro avg       0.66      0.66      0.66      1384
weighted avg       0.66      0.66      0.66      1384

Corss Validation Score -  60.954
Testing Data
Accuracy -  66.01
              precision    recall  f1-score   support

           0       0.65      0.68      0.67       912
           1       0.67      0.64      0.65       909

    accuracy                           0.66      1821
   macro avg       0.66      0.66      0.66      1821
weighted avg       0.66      0.66      0.66      1821



In [ ]:
#RandomForest- Classifier

rf_model = RandomForestClassifier()
rf_model.fit(x_train_data, y_train_data)
validation_predicted_rf = rf_model.predict(x_validation_data)
validation_accuracy_rf = round(accuracy_score(validation_predicted_rf, y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_rf)
print(classification_report(y_validation_data, validation_predicted_rf))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(rf_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_rf = rf_model.predict(x_test_data)
test_accuracy_rf = round(accuracy_score(test_predicted_rf, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_rf)
print(classification_report(y_test_data, test_predicted_rf))

Validation Data
Accuracy -  72.25
              precision    recall  f1-score   support

           0       0.74      0.69      0.71       695
           1       0.71      0.76      0.73       689

    accuracy                           0.72      1384
   macro avg       0.72      0.72      0.72      1384
weighted avg       0.72      0.72      0.72      1384

Corss Validation Score -  65.733
Testing Data
Accuracy -  73.31
              precision    recall  f1-score   support

           0       0.74      0.72      0.73       912
           1       0.73      0.75      0.74       909

    accuracy                           0.73      1821
   macro avg       0.73      0.73      0.73      1821
weighted avg       0.73      0.73      0.73      1821



In [ ]:
#XGB-Classifier

xgb_model = XGBClassifier()
xgb_model.fit(x_train_data, y_train_data)
validation_predicted_xgb = xgb_model.predict(x_validation_data)
validation_accuracy_xgb = round(accuracy_score(validation_predicted_xgb, y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_xgb)
print(classification_report(y_validation_data, validation_predicted_xgb))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(xgb_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_xgb = xgb_model.predict(x_test_data)
test_accuracy_xgb = round(accuracy_score(test_predicted_xgb, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_xgb)
print(classification_report(y_test_data, test_predicted_xgb))

Validation Data
Accuracy -  61.78
              precision    recall  f1-score   support

           0       0.75      0.35      0.48       695
           1       0.58      0.88      0.70       689

    accuracy                           0.62      1384
   macro avg       0.67      0.62      0.59      1384
weighted avg       0.67      0.62      0.59      1384

Corss Validation Score -  62.052
Testing Data
Accuracy -  63.21
              precision    recall  f1-score   support

           0       0.76      0.39      0.51       912
           1       0.59      0.88      0.70       909

    accuracy                           0.63      1821
   macro avg       0.67      0.63      0.61      1821
weighted avg       0.67      0.63      0.61      1821



In [ ]:
Accuracies:

MultinomialNB - 78.75%
SVM - 78.47%
KNN - 60.85%
Decision Tree - 66.01%
Random Forest - 73.31%
XG Boost - 63.21%